In [7]:

import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer


import matplotlib.pyplot as plt


In [8]:
df = pd.read_csv('diabetesData.csv')
display(df.head())
print(df.shape)
df.info()
df.describe().T

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


(768, 9)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


,count,mean,std,min,25%,50%,75%,max
Pregnancies,768.0,3.845052,3.369578,0.000,1.00000,3.0000,6.00000,17.00
Glucose,768.0,120.894531,31.972618,0.000,99.00000,117.0000,140.25000,199.00
BloodPressure,768.0,69.105469,19.355807,0.000,62.00000,72.0000,80.00000,122.00
SkinThickness,768.0,20.536458,15.952218,0.000,0.00000,23.0000,32.00000,99.00
Insulin,768.0,79.799479,115.244002,0.000,0.00000,30.5000,127.25000,846.00
BMI,768.0,31.992578,7.884160,0.000,27.30000,32.0000,36.60000,67.10
DiabetesPedigreeFunction,768.0,0.471876,0.331329,0.078,0.24375,0.3725,0.62625,2.42
Age,768.0,33.240885,11.760232,21.000,24.00000,29.0000,41.00000,81.00
Outcome,768.0,0.348958,0.476951,0.000,0.00000,0.0000,1.00000,1.00


In [9]:
print("Duplicados:", df.duplicated().sum())
df = df.drop_duplicates()
print("Nuevo shape:", df.shape)


Duplicados: 0
Nuevo shape: (768, 9)


In [ ]:
cols_invalid_zero = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']

# Contar ceros por columna
print("Ceros por columna:")
print((df[cols_invalid_zero] == 0).sum())

Ceros por columna:
Glucose            5
BloodPressure     35
SkinThickness    227
Insulin          374
BMI               11
dtype: int64


In [11]:
df[cols_invalid_zero] = df[cols_invalid_zero].replace(0, np.nan) #Reemplaza ceros 
print("Nulos por columna (después):")
print(df.isnull().sum())

Nulos por columna (después):
Pregnancies                   0
Glucose                       5
BloodPressure                35
SkinThickness               227
Insulin                     374
BMI                          11
DiabetesPedigreeFunction      0
Age                           0
Outcome                       0
dtype: int64


In [12]:
imputer = SimpleImputer(strategy='median') #Insertar meidana en los valores vacíos
df_imputed = df.copy() #Copia de base de datos con valores rellenos
df_imputed[cols_invalid_zero] = imputer.fit_transform(df_imputed[cols_invalid_zero]) 
print("Medianas antes (con NaN):")
print(df[cols_invalid_zero].median())

print("Medianas después (imputadas):")
print(pd.DataFrame(df_imputed[cols_invalid_zero]).median())


Medianas antes (con NaN):
Glucose          117.0
BloodPressure     72.0
SkinThickness     29.0
Insulin          125.0
BMI               32.3
dtype: float64
Medianas después (imputadas):
Glucose          117.0
BloodPressure     72.0
SkinThickness     29.0
Insulin          125.0
BMI               32.3
dtype: float64


In [16]:
def detect_outliers_iqr(series, k=1.5):
    q1 = series.quantile(0.25)    # primer cuartil (25%)
    q3 = series.quantile(0.75)    # tercer cuartil (75%)
    iqr = q3 - q1                 # rango intercuartílico (IQR)
    lower = q1 - k * iqr          # límite inferior
    upper = q3 + k * iqr          # límite superior
    return series[(series < lower) | (series > upper)]

out_bmi = detect_outliers_iqr(df_imputed['BMI'])
print("Outliers BMI (count):", out_bmi.count())

Outliers BMI (count): 8


In [17]:
#Eliminación de los valores fuera den rango
df_capped = df_imputed.copy() 
for col in ['Glucose','BloodPressure','SkinThickness','Insulin','BMI','Age','Pregnancies','DiabetesPedigreeFunction']:
    lower = df_capped[col].quantile(0.01)
    upper = df_capped[col].quantile(0.99)
    df_capped[col] = df_capped[col].clip(lower, upper)


In [18]:
# Definimos columnas numéricas
numeric_features = ['Pregnancies','Glucose','BloodPressure','SkinThickness',
                    'Insulin','BMI','DiabetesPedigreeFunction','Age']

# Pipeline numérico: imputar (median) + escalar.
# Usa RobustScaler si hay muchos outliers; StandardScaler si distribución aproxim. normal
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())   # o RobustScaler()
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features)
    ])

# Aplicarlo y obtener X_preprocessed (numpy)
X_preprocessed = preprocessor.fit_transform(df[numeric_features])

# Convertir a DataFrame manteniendo nombres de columnas
X_preprocessed_df = pd.DataFrame(X_preprocessed, columns=numeric_features)
X_preprocessed_df.head()


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,0.639947,0.866045,-0.031990,0.670643,-0.181541,0.166619,0.468492,1.425995
1,-0.844885,-1.205066,-0.528319,-0.012301,-0.181541,-0.852200,-0.365061,-0.190672
2,1.233880,2.016662,-0.693761,-0.012301,-0.181541,-1.332500,0.604397,-0.105584
3,-0.844885,-1.073567,-0.528319,-0.695245,-0.540642,-0.633881,-0.920763,-1.041549
4,-1.141852,0.504422,-2.679076,0.670643,0.316566,1.549303,5.484909,-0.020496


In [20]:
y = df['Outcome'].copy()  # etiqueta original (sin modificar)
preprocessed_full = pd.concat([X_preprocessed_df, y.reset_index(drop=True)], axis=1)

# Guardar a CSV para inspección posterior (opcional)
preprocessed_full.to_csv('diabetes_preprocessed.csv', index=False)
print("Preprocesado guardado como diabetes_preprocessed.csv")
preprocessed_full.head()

Preprocesado guardado como diabetes_preprocessed.csv


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,0.639947,0.866045,-0.031990,0.670643,-0.181541,0.166619,0.468492,1.425995,1
1,-0.844885,-1.205066,-0.528319,-0.012301,-0.181541,-0.852200,-0.365061,-0.190672,0
2,1.233880,2.016662,-0.693761,-0.012301,-0.181541,-1.332500,0.604397,-0.105584,1
3,-0.844885,-1.073567,-0.528319,-0.695245,-0.540642,-0.633881,-0.920763,-1.041549,0
4,-1.141852,0.504422,-2.679076,0.670643,0.316566,1.549303,5.484909,-0.020496,1


In [21]:
# Separar datos en entrenamiento y prueba
from sklearn.model_selection import train_test_split
X = preprocessed_full.drop('Outcome', axis=1)
y = preprocessed_full['Outcome']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Tamaño total:", X.shape)
print("Entrenamiento:", X_train.shape, y_train.shape)
print("Prueba:", X_test.shape, y_test.shape)


Tamaño total: (768, 8)
Entrenamiento: (614, 8) (614,)
Prueba: (154, 8) (154,)


In [ ]:
from sklearn.linear_model import LogisticRegression

modelo = LogisticRegression(max_iter=1000)  # le damos más iteraciones para que aprenda bien

modelo.fit(X_train, y_train)

✅ Modelo entrenado con éxito


In [23]:
# Usamos el modelo entrenado para predecir en los datos de prueba
y_pred = modelo.predict(X_test)

# primeras 10 predicciones
print("Predicciones del modelo (primeros 10 pacientes de prueba):")
print(y_pred[:10])

#respuestas reales de esos mismos pacientes
print("Valores reales (primeros 10):")
print(y_test.values[:10])

Predicciones del modelo (primeros 10 pacientes de prueba):
[0 0 0 0 0 0 0 0 1 1]
Valores reales (primeros 10):
[0 0 0 0 0 0 0 0 0 0]


In [25]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

#Accuracy (qué porcentaje acierta)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy del modelo:", accuracy)

#Matriz de confusión (cuenta aciertos y errores)
print("Matriz de confusión:")
print(confusion_matrix(y_test, y_pred))

#Reporte completo con precision, recall y f1-score
print("Reporte de clasificación:")
print(classification_report(y_test, y_pred))

Accuracy del modelo: 0.7532467532467533
Matriz de confusión:
[[82 17]
 [21 34]]
Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.80      0.83      0.81        99
           1       0.67      0.62      0.64        55

    accuracy                           0.75       154
   macro avg       0.73      0.72      0.73       154
weighted avg       0.75      0.75      0.75       154

